I should really be using the catalog for all this, since that is what I wrote it for.

In [1]:
from antelope_catalog import LcCatalog
cat = LcCatalog('/data/GitHub/2018/bw2-uncertainty/catalog/')

Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/flowable_synlist.json:
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/quantity_synlist.json:
local.qdb: /data/GitHub/2018/bw2-uncertainty/catalog/reference-quantities.json
Loading JSON data from /data/GitHub/2018/bw2-uncertainty/catalog/reference-quantities.json:
local.qdb: /data/GitHub/lca-tools/lcatools/providers/data/elcd_reference_quantities.json
24 new quantity entities added (24 total)
5 new flow entities added (5 total)


## Step 1: Install Ecoinvent 3.4

In [2]:
from antelope_catalog.data_sources.ecoinvent import EcoinventConfig

In [3]:
ec = EcoinventConfig('/data/LCI/Ecoinvent')

In [4]:
for ref in ec.references:
    if ref.find('3.4') > 0:
        for res in ec.make_resources(ref):
            cat.add_resource(res)

In [5]:
[x for x in cat.interfaces]

['local.ecoinvent.3.4.apos:basic',
 'local.ecoinvent.3.4.apos:index',
 'local.ecoinvent.3.4.apos:inventory',
 'local.ecoinvent.3.4.conseq:basic',
 'local.ecoinvent.3.4.conseq:index',
 'local.ecoinvent.3.4.conseq:inventory',
 'local.ecoinvent.3.4.cutoff:basic',
 'local.ecoinvent.3.4.cutoff:index',
 'local.ecoinvent.3.4.cutoff:inventory',
 'local.ecoinvent.3.4.cutoff.index__20180905:background',
 'local.ecoinvent.3.4.cutoff.index__20180905:basic',
 'local.ecoinvent.3.4.cutoff.index__20180905:index',
 'local.qdb:basic',
 'local.qdb:index',
 'local.qdb:quantity']

In [6]:
# trigger data source index + background creation
cat.query('local.ecoinvent.3.4.cutoff').foreground_flows()

local.ecoinvent.3.4.cutoff.index__20180905: /data/GitHub/2018/bw2-uncertainty/catalog/archives/local.ecoinvent.3.4.cutoff.index__20180905_background.mat
Loading JSON data from /data/GitHub/2018/bw2-uncertainty/catalog/index/ffc0f70037f9e921067e85ac7e2d4ff0ba1bc3b5.json.gz:
local.ecoinvent.3.4.cutoff.index__20180905: /data/GitHub/2018/bw2-uncertainty/catalog/index/ffc0f70037f9e921067e85ac7e2d4ff0ba1bc3b5.json.gz
local.ecoinvent.3.4.cutoff: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02
19 new quantity entities added (19 total)
13459 new process entities added (13459 total)
4881 new flow entities added (4881 total)
local.ecoinvent.3.4.cutoff: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02
Squashing bad termination in linked reference exchange, c72948d5-838a-4cb8-82ab-20d600051e93
Flow d3d23054-a030-4284-9cfb-c464e70ec865 Term cca9f6a1-facd-423d-8984-168c6c4d570b
Squashing bad termination in linked reference exchange, 8b39afef-2119-46c4-83cb-db8fe05c0cd5
Flow 2c9486c9-1f

<generator object TarjanBackgroundImplementation.foreground_flows at 0x7f32146e7308>

## LCIA

In [ ]:
cat.new_resource('local.lcia.ecoinvent.3.1.2014_08_13', '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx',
                'EcoinventLcia', interfaces=('index', 'quantity'), static=True, 
                 ns_uuid='46802ca5-8b25-398c-af10-2376adaa4623')